In [65]:
# OUVRIR LE CSV
import pandas as pd
import numpy as np
df = pd.read_csv('Pfitzer10-100.csv', sep = ";")

# Replace comma with dot and convert to float
df['x'] = pd.to_numeric(df['x'].str.replace(',', '.'), errors='coerce')
df['y'] = pd.to_numeric(df['y'].str.replace(',', '.'), errors='coerce')
df['workload index'] = pd.to_numeric(df['workload index'].str.replace(',', '.'), errors='coerce')

df_for_fixed_SR = df[df['current office'] == 1]

clusters = {
    str(i + 1): df_for_fixed_SR[df_for_fixed_SR[str(i + 1)] == 1][['x', 'y']].values
    for i in range(len(df_for_fixed_SR))
}

# Calculer les distances pour chaque cluster
for cluster_id, points in clusters.items():
    distances = np.linalg.norm(df[['x', 'y']].values[:, None] - points, axis=2)
    
    df[f'distance_to_{cluster_id}'] = distances
    
Dist_Matrix = df.filter(like='distance_to_').to_numpy().T

WL_Matrix = df.filter(like='workload index').to_numpy()
initial_bricks = {
    i: {
        "center_brick": df_for_fixed_SR[df_for_fixed_SR[str(i + 1)] == 1]['zone'].values[0],
        "assigned_bricks": df[df[str(i + 1)] == 1]['zone'].to_numpy() - 1
    }
    for i in range(len(df_for_fixed_SR))
}

initial_bricks


{0: {'center_brick': 3,
  'assigned_bricks': array([ 0,  1,  2,  3, 10, 11, 12, 13, 21, 22])},
 1: {'center_brick': 18,
  'assigned_bricks': array([ 4,  5,  6,  7,  8,  9, 14, 15, 16, 17, 18, 25])},
 2: {'center_brick': 30,
  'assigned_bricks': array([19, 26, 27, 28, 29, 37, 38, 39, 48, 49, 73])},
 3: {'center_brick': 33,
  'assigned_bricks': array([20, 23, 24, 30, 31, 32, 33, 40, 41])},
 4: {'center_brick': 59,
  'assigned_bricks': array([34, 35, 36, 42, 43, 44, 45, 46, 47, 57, 58, 59])},
 5: {'center_brick': 82,
  'assigned_bricks': array([50, 51, 60, 61, 70, 71, 80, 81, 90])},
 6: {'center_brick': 73,
  'assigned_bricks': array([52, 53, 62, 72, 82, 83, 91, 92, 93, 94])},
 7: {'center_brick': 66,
  'assigned_bricks': array([54, 55, 56, 63, 64, 65, 66, 74])},
 8: {'center_brick': 79,
  'assigned_bricks': array([67, 68, 69, 76, 77, 78, 79, 88, 89])},
 9: {'center_brick': 98,
  'assigned_bricks': array([75, 84, 85, 86, 87, 95, 96, 97, 98, 99])}}

In [ ]:
from gurobipy import *





def epsilon_constraint(
    Dist_Matrix, WL_Matrix, workload_range, epsilon, epsilon2, initial_bricks
):
    m = Model("brick")
    assert Dist_Matrix.shape[1] == WL_Matrix.shape[0], "Mismatch in dimensions of Dist_Matrix and WL_Matrix"
    assert len(initial_bricks) == Dist_Matrix.shape[0], "Mismatch in dimensions of Dist_Matrix and initial_bricks"
    # Create variables
    num_bricks = Dist_Matrix.shape[1]
    num_groups = Dist_Matrix.shape[0]
    print(num_bricks, num_groups)

    x = m.addVars(num_bricks, num_groups, vtype=GRB.BINARY, name="x")
    max_new_value = m.addVar(vtype=GRB.CONTINUOUS, name="max_new_value")
    z = m.addVar(name="z")
    
    # Set constraints
    for j in range(num_groups):
        
        new_brick_value = sum(
            WL_Matrix[brick] * x[brick, j]
            for brick in range(num_bricks)
            if brick + 1 not in initial_bricks[j]["assigned_bricks"]
        )
        m.addConstr(max_new_value >= new_brick_value[0])

    m.addConstrs(
        (sum(x[i, j] for j in range(num_groups)) == 1 for i in range(num_bricks))
    )
    m.addConstrs(
        (
            sum(WL_Matrix[i][0] * x[i, j] for i in range(num_bricks)) <= workload_range[1]
            for j in range(num_groups)
        )
    )
    m.addConstrs(
        (
            sum(WL_Matrix[i][0] * x[i, j] for i in range(num_bricks)) >= workload_range[0]
            for j in range(num_groups)
        )
    )

    m.addConstrs(
        (
            z >= sum(Dist_Matrix[j, i] * x[i, j] for i in range(num_bricks))
            for j in range(num_groups)
        )
    )

    m.addConstr(max_new_value <= epsilon2)

    # Set objective
    m.setObjective(z + epsilon * max_new_value, GRB.MINIMIZE)

    # Optimize the model
    m.optimize()

    return m, z, max_new_value


def go_epsilon_constraint(
    Dist_Matrix, WL_Matrix, initial_bricks, epsilon, workload_range
):
    epsilon2 = np.inf
    list_model = []
    m, z, max_new_value = epsilon_constraint(
        Dist_Matrix, WL_Matrix, workload_range, epsilon, epsilon2, initial_bricks
    )
    while m.status == GRB.OPTIMAL:
        epsilon2 = max_new_value.X - 0.001
        list_model.append((z.X, max_new_value.X))
        m, z, max_new_value = epsilon_constraint(
            Dist_Matrix, WL_Matrix, workload_range, epsilon, epsilon2, initial_bricks
        )

    return list_model


workload_ranges = [(0.8, 1.2), (0.85, 1.15), (0.9, 1.1)]
for workload_range in workload_ranges:
    list = go_epsilon_constraint(
        Dist_Matrix, WL_Matrix, initial_bricks, 0.01, workload_range
    )

    import matplotlib.pyplot as plt

    # Extract z and max_new_value from the list
    z_values, max_new_values = zip(*list)

    # Plot the values
    plt.plot(z_values, max_new_values, marker="o")
    plt.xlabel("z (Objective Value)")
    plt.ylabel("max_new_value")
    plt.title("Epsilon Constraint Method Results")
    plt.grid(True)
plt.show()

100 10
[0.0536]


GurobiError: Constraint has no bool value (are you trying "lb <= expr <= ub"?)